In [ ]:
%%capture
!pip install folium==0.2.1
!pip install pdflatex
!sudo apt update
!sudo apt install -y texlive-latex-recommended 
!sudo apt install -y texlive-latex-extra
!sudo apt install -y dvipng
!mkdir ./drive/MyDrive/darts_graphs
!mkdir ./drive/MyDrive/needles_graphs

In [ ]:
import tarfile
import pandas as pd
import io
import matplotlib.pyplot as plt
from scipy import stats
import math
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

---
# Darts experiment. Graphs

In [ ]:
experiment="darts"
tar = tarfile.open('./drive/MyDrive/'+experiment+'.tar.gz','r:gz')

## Points Correlation

In [ ]:
points = pd.read_csv(io.TextIOWrapper(tar.extractfile('points.csv'), encoding='utf-8'), delimiter=',')

In [ ]:
points.hist(edgecolor='black', linewidth=1.2)
plt.savefig('drive/MyDrive/'+experiment+'_graphs/histogram.pgf')

In [ ]:
table = [[0 for _ in range(10)] for _ in range(10)]
for index, point in tqdm(points.iterrows()):
  X = int(math.trunc(point['X']*10))
  Y = int(math.trunc(point[' Y']*10))
  if X == 10:
    X = 9
  if Y == 10:
    Y = 9
  
  table[X][Y] += 1
print(table)

Backup of table: 

```
table = [[1745145, 1743955, 1745232, 1744564, 1746407, 1744605, 1746542, 1746672, 1744476, 1743516],
         [1745270, 1746324, 1748509, 1744806, 1746836, 1745933, 1745279, 1742781, 1746230, 1744721],
         [1747144, 1745078, 1745527, 1746533, 1747825, 1745591, 1745113, 1748146, 1745903, 1748439],
         [1743395, 1743625, 1747289, 1747116, 1746891, 1745744, 1746801, 1744843, 1748323, 1744737],
         [1746659, 1746695, 1746828, 1745925, 1745895, 1743369, 1745108, 1748394, 1742910, 1747143],
         [1745416, 1746147, 1747978, 1746176, 1745469, 1746341, 1745014, 1746914, 1746220, 1744766],
         [1746182, 1748644, 1747573, 1745971, 1746631, 1744745, 1743806, 1747284, 1746923, 1745694],
         [1744587, 1746206, 1747853, 1746838, 1746575, 1746781, 1744927, 1747990, 1745125, 1746856],
         [1745560, 1747070, 1745350, 1745748, 1747400, 1745993, 1747877, 1746417, 1747316, 1745785],
         [1744578, 1743407, 1745620, 1744546, 1747630, 1744052, 1745102, 1747261, 1744886, 1745353]]

```



In [ ]:
total = sum([sum(i) for i in table])
print(total)

In [ ]:
ranges = ['0.0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9']
a = "\\begin{table}[]\n\\begin{tabular}{l|"+len(table[0])*"l|"+"}\n \cline{2-11} \n X \ Y & " + " & ".join(ranges) + " \\\\ \\hline \n"
for i in range(len(ranges)):
  a += "\\multicolumn{1}{|l|}{" + ranges[i] + "} & " + " & ".join([str(c) for c in table[i]]) + " \\\\ \\hline \n"
a += "\\end{tabular} \n\\end{table}"
print(a)


In [ ]:
del points
del table
del total
del ranges
del a

## Approximation evolution

In [ ]:
evolution = pd.read_csv(io.TextIOWrapper(tar.extractfile('evolution.csv'), encoding='utf-8'), header=None, delimiter=',')[0].to_numpy()
OBJ = math.pi/4

In [ ]:
unq = [evolution[0]]
idx = [0]
for i in tqdm(range(len(evolution))):
  if evolution[i] != unq[-1]:
    unq.append(evolution[i])
    idx.append(i)
print(len(unq))

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.plot(idx, unq)
ax.axhline(y=OBJ, linestyle='dashed', color='red')
ax.set_xlabel('Iteration')
ax.set_ylabel('Approximation')
ax.set_xscale('log')
fig.savefig('drive/MyDrive/'+experiment+'_graphs/approximation_unique.pgf')

In [ ]:
ABS = []
for i in tqdm(unq):
  ABS.append(abs(OBJ-i))
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.plot(idx, ABS)
ax.set_xlabel('Iteration')
ax.set_ylabel('Absolute Error')
ax.grid(True)
ax.set_xscale('log')
fig.savefig('drive/MyDrive/'+experiment+'_graphs/absolute_error.pgf')

In [ ]:
percentage = []
for i in tqdm(unq):
  percentage.append(i/OBJ - 1)
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.grid(True)
ax.plot(idx, percentage)
ax.set_xlabel('Iteration')
ax.set_ylabel('Percentage Error')
ax.set_xscale('log')
fig.savefig('drive/MyDrive/darts_graphs/error_percentage.pgf')

In [ ]:
def similarity(a, b):
  a_str = str(a)
  b_str = str(b)
  count = 0
  while(count < len(a_str) and count < len(b_str) and a_str[count] == b_str[count]):
    count += 1
  return count - 2

In [ ]:
precission = []
showed = False
for p in tqdm(unq):
  precission.append(similarity(p, OBJ) )
  if not showed and precission[-1] > 5:
    print(len(precission))
    showed = True

fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.grid(True)
ax.plot(idx, precission)
ax.set_xlabel('Iteration')
ax.set_ylabel('Decimal precission')
ax.set_xscale('log')
fig.savefig('drive/MyDrive/'+experiment+'_graphs/decimal_precission.pgf')

In [ ]:
SE = []
for i in tqdm(unq):
  SE.append((OBJ-i)**2)
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.plot(idx, SE)
ax.set_xlabel('Iteration')
ax.set_ylabel('Squared Error')
ax.grid(True)
ax.set_xscale('log')
fig.savefig('drive/MyDrive/darts_graphs/squared_error.pgf')

In [ ]:
logA = np.log10(idx[1:])
logB = np.log10(SE[1:])
coefficients = np.polyfit(logA, logB, 1)
polynomial = np.poly1d(coefficients)
ys = polynomial(logA)

fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.plot(logA, logB, label="Logarithm of Squared Error")
ax.plot(logA, ys, label="m="+str(coefficients[0]))
ax.legend()
fig.savefig('drive/MyDrive/'+experiment+'_graphs/loglog_approx.pgf')

In [ ]:
#Preprocessing
ZOOM_LEVEL = 10**5
evolution = evolution[ZOOM_LEVEL:]
unq = [evolution[0]]
idx = [ZOOM_LEVEL]
for i in tqdm(range(len(evolution))):
  if evolution[i] != unq[-1]:
    unq.append(evolution[i])
    idx.append(ZOOM_LEVEL+i)
print(len(unq))

#Absolute error
ABS = []
for i in tqdm(unq):
  ABS.append(abs(OBJ-i))
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.plot(idx, ABS)
ax.set_xlabel('Iteration')
ax.set_ylabel('Absolute Error')
ax.grid(True)
ax.set_xscale('log')
fig.savefig('drive/MyDrive/'+experiment+'_graphs/absolute_error_zoom.pgf')

#Squared error
SE = []
for i in tqdm(unq):
  SE.append((OBJ-i)**2)
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.plot(idx, SE)
ax.set_xlabel('Iteration')
ax.set_ylabel('Squared Error')
ax.grid(True)
ax.set_xscale('log')
fig.savefig('drive/MyDrive/'+experiment+'_graphs/squared_error_zoom.pgf')

#Percentage error
percentage = []
for i in tqdm(unq):
  percentage.append(i/OBJ - 1)
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.grid(True)
ax.plot(idx, percentage)
ax.set_xlabel('Iteration')
ax.set_ylabel('Percentage Error')
ax.set_xscale('log')
fig.savefig('drive/MyDrive/'+experiment+'_graphs/error_percentage_zoom.pgf')

#Decimal precission
precission = []
showed = False
for p in tqdm(unq):
  precission.append(similarity(p, OBJ) )
  if not showed and precission[-1] > 5:
    print(len(precission))
    showed = True

fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.grid(True)
ax.plot(idx, precission)
ax.set_xlabel('Iteration')
ax.set_ylabel('Decimal precission')
ax.set_xscale('log')
fig.savefig('drive/MyDrive/'+experiment+'_graphs/decimal_precission_zoom.pgf')

In [ ]:
del fig
del ax
del unq
del idx
del SE
del ABS
del precission
del showed
del percentage

---
# Needle's buffon experiment. Graphs






In [ ]:
experiment="needles"
tar = tarfile.open('./drive/MyDrive/'+experiment+'.tar.gz','r:gz')

## Points Correlation

In [ ]:
points = pd.read_csv(io.TextIOWrapper(tar.extractfile('points.csv'), encoding='utf-8'), delimiter=',')

In [ ]:
points.hist(edgecolor='black', linewidth=1.2)
plt.savefig('drive/MyDrive/'+experiment+'_graphs/histogram.pgf')

In [ ]:
table = [[0 for _ in range(10)] for _ in range(10)]
alpha = 20/math.pi
for index, point in tqdm(points.iterrows()):
  X = int(math.trunc(point['X']*20))
  Y = int(math.trunc(alpha*point['Theta']))
  if X == 10:
    X = 9
  if Y == 10:
    Y = 9
  
  table[X][Y] += 1
print(table)

Backup of table: 

```
table = [[3494474, 3492729, 3490494, 3493320, 3494240, 3489546, 3490327, 3492164, 3487630, 3490141], 
         [3490156, 3492111, 3489345, 3494815, 3491652, 3491365, 3490987, 3491057, 3490720, 3490946], 
         [3491301, 3493562, 3495663, 3488261, 3491413, 3491371, 3493552, 3492123, 3493766, 3491710], 
         [3490484, 3487447, 3491976, 3492694, 3491445, 3492973, 3493845, 3490717, 3494012, 3493776], 
         [3489295, 3497169, 3491152, 3492070, 3491482, 3491884, 3493455, 3493747, 3494042, 3490587], 
         [20, 17, 15, 12, 24, 22, 13, 21, 22, 15], 
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
```



In [ ]:
total = sum([sum(i) for i in table])
print(total)

In [ ]:
rangeX = ['%.1f'%(0.05*i) for i in range(10)]
rangeT = ['%.5f'%((math.pi/20)*i) for i in range(10)]
a = "\\begin{table}[]\n\\begin{tabular}{l|"+len(table[0])*"l|"+"}\n \cline{2-11} \n X \ $\\theta$ & " + " & ".join(rangeT) + " \\\\ \\hline \n"
for i in range(len(rangeX)):
  a += "\\multicolumn{1}{|l|}{" + rangeX[i] + "} & " + " & ".join([str(c) for c in table[i]]) + " \\\\ \\hline \n"
a += "\\end{tabular} \n\\end{table}"
print(a)

In [ ]:
del points
del table
del total
del rangeX
del rangeT
del a

## Approximation Evolution

In [ ]:
evolution = pd.read_csv(io.TextIOWrapper(tar.extractfile('evolution.csv'), encoding='utf-8'), header=None, delimiter=',')[0].to_numpy()
OBJ = 1/math.pi

In [ ]:
unq = [evolution[0]]
idx = [0]
for i in tqdm(range(len(evolution))):
  if evolution[i] != unq[-1]:
    unq.append(evolution[i])
    idx.append(i)
print(len(unq))

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.plot(idx, unq)
ax.axhline(y=OBJ, linestyle='dashed', color='red')
ax.set_xlabel('Iteration')
ax.set_ylabel('Approximation')
ax.set_xscale('log')
fig.savefig('drive/MyDrive/'+experiment+'_graphs/approximation_unique.pgf')

In [ ]:
ABS = []
for i in tqdm(unq):
  ABS.append(abs(OBJ-i))
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.plot(idx, ABS)
ax.set_xlabel('Iteration')
ax.set_ylabel('Absolute Error')
ax.grid(True)
ax.set_xscale('log')
fig.savefig('drive/MyDrive/'+experiment+'_graphs/absolute_error.pgf')

In [ ]:
percentage = []
for i in tqdm(unq):
  percentage.append(i/OBJ - 1)
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.grid(True)
ax.plot(idx, percentage)
ax.set_xlabel('Iteration')
ax.set_ylabel('Error Percentage')
ax.set_xscale('log')
fig.savefig('drive/MyDrive/'+experiment+'_graphs/error_percentage.pgf')

In [ ]:
def similarity(a, b):
  a_str = str(a)
  b_str = str(b)
  count = 0
  while(count < len(a_str) and count < len(b_str) and a_str[count] == b_str[count]):
    count += 1
  return count - 2

In [ ]:
precission = []
showed = False
for p in tqdm(unq):
  precission.append(similarity(p, OBJ) )
  if not showed and precission[-1] > 5:
    print(len(precission))
    showed = True

fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.grid(True)
ax.plot(idx, precission)
ax.set_xlabel('Iteration')
ax.set_ylabel('Decimal precission')
ax.set_xscale('log')
fig.savefig('drive/MyDrive/'+experiment+'_graphs/decimal_precission.pgf')

In [ ]:
SE = []
for i in tqdm(unq):
  SE.append((OBJ-i)**2)
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.plot(idx, SE)
ax.set_xlabel('Iteration')
ax.set_ylabel('Squared Error')
ax.grid(True)
ax.set_xscale('log')
fig.savefig('drive/MyDrive/'+experiment+'_graphs/squared_error.pgf')

In [ ]:
logA = np.log10(idx[1:])
logB = np.log10(SE[1:])
coefficients = np.polyfit(logA, logB, 1)
polynomial = np.poly1d(coefficients)
ys = polynomial(logA)

fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.plot(logA, logB, label="Logarithm of Squared Error")
ax.plot(logA, ys, label="m="+str(coefficients[0]))
ax.legend()
fig.savefig('drive/MyDrive/'+experiment+'_graphs/loglog_approx.pgf')

In [ ]:
#Preprocessing
ZOOM_LEVEL = 10**5
evolution = evolution[ZOOM_LEVEL:]
unq = [evolution[0]]
idx = [10**5]
for i in tqdm(range(len(evolution))):
  if evolution[i] != unq[-1]:
    unq.append(evolution[i])
    idx.append(10**5+i)
print(len(unq))

#Absolute error
ABS = []
for i in tqdm(unq):
  ABS.append(abs(OBJ-i))
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.plot(idx, ABS)
ax.set_xlabel('Iteration')
ax.set_ylabel('Absolute Error')
ax.grid(True)
ax.set_xscale('log')
fig.savefig('drive/MyDrive/'+experiment+'_graphs/absolute_error_zoom.pgf')

#Squared error
SE = []
for i in tqdm(unq):
  SE.append((OBJ-i)**2)
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.plot(idx, SE)
ax.set_xlabel('Iteration')
ax.set_ylabel('Squared Error')
ax.grid(True)
ax.set_xscale('log')
fig.savefig('drive/MyDrive/'+experiment+'_graphs/squared_error_zoom.pgf')

#Percentage error
percentage = []
for i in tqdm(unq):
  percentage.append(i/OBJ - 1)
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.grid(True)
ax.plot(idx, percentage)
ax.set_xlabel('Iteration')
ax.set_ylabel('Percentage Error')
ax.set_xscale('log')
fig.savefig('drive/MyDrive/'+experiment+'_graphs/error_percentage_zoom.pgf')

#Decimal precission
precission = []
showed = False
for p in tqdm(unq):
  precission.append(similarity(p, OBJ) )
  if not showed and precission[-1] > 5:
    print(len(precission))
    showed = True

fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.grid(True)
ax.plot(idx, precission)
ax.set_xlabel('Iteration')
ax.set_ylabel('Decimal precission')
ax.set_xscale('log')
fig.savefig('drive/MyDrive/'+experiment+'_graphs/decimal_precission_zoom.pgf')

In [ ]:
del fig
del ax
del unq
del precission
del idx
del showed
del ABS
del percentage
del SE

---
# Comparison

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.75,0.75])
ax.set_xlabel('Iteration')
ax.set_ylabel('Normalized absolute error')

experiment="darts"
tar = tarfile.open('./drive/MyDrive/'+experiment+'.tar.gz','r:gz')
evolution = pd.read_csv(io.TextIOWrapper(tar.extractfile('evolution.csv'), encoding='utf-8'), header=None, delimiter=',')[0].to_numpy()
OBJ = math.pi/4
unq = [evolution[0]]
percentage = [abs(evolution[0]-OBJ)/OBJ]
idx = [1]
for i in tqdm(range(len(evolution))):
  if evolution[i] != unq[-1]:
    unq.append(evolution[i])
    percentage.append(abs(evolution[i]-OBJ)/OBJ)
    idx.append(i+1)


ax.loglog(idx,percentage, label='Darts\' experiment error')
logA = np.log10(idx)
logB = np.log10(percentage)
coefficients = np.polyfit(logA, logB, 1)
polynomial = np.poly1d(coefficients)
ys = polynomial(logA)
ax.plot(10**logA, 10**ys, label="Darts\' experiment m=%.2f"%(coefficients[0]))

experiment="needls"
tar = tarfile.open('./drive/MyDrive/'+experiment+'.tar.gz','r:gz')
evolution = pd.read_csv(io.TextIOWrapper(tar.extractfile('evolution.csv'), encoding='utf-8'), header=None, delimiter=',')[0].to_numpy()
OBJ = 1/math.pi
unq = [evolution[0]]
percentage = [abs(evolution[0]-OBJ)/OBJ]
idx = [1]
for i in tqdm(range(len(evolution))):
  if evolution[i] != unq[-1]:
    unq.append(evolution[i])
    percentage.append(abs(evolution[i]-OBJ)/OBJ)
    idx.append(i+1)

ax.loglog(idx,percentage, label='Needles\' experiment error')
logA = np.log10(idx)
logB = np.log10(percentage)
coefficients = np.polyfit(logA, logB, 1)
polynomial = np.poly1d(coefficients)
ys = polynomial(logA)
ax.plot(10**logA, 10**ys, label="Needles\' experiment m=%.2f"%(coefficients[0]))

ax.legend()
fig.savefig('drive/MyDrive/comparison.pgf')